# Gold Prediction SubModel Training - real_rate Attempt 9

**Method**: Real Yield Curve Shape + Slope-Curvature Interaction (5-Feature Design)

**Change from Attempt 7**: All 4 attempt-7 features retained + 5th interaction feature added.
- Attempt 7 (4 feat incl rr_slope_level_z): Gate 3 PASS via Sharpe (+0.329), DA fail (-0.00149pp)
- Attempt 8 (3 feat excl rr_slope_level_z): Gate 3 PASS via MAE (-0.0203%), Sharpe fail (-0.260)
- Attempt 9 goal: retain all 4 attempt-7 features (preserve Sharpe) + add slope-curvature interaction
  (low-autocorr, orthogonal) to push DA into positive territory.

**New Feature**: rr_slope_curvature_interaction_z = rolling_zscore(slope.diff() * curvature.diff(), 60)
- Captures simultaneous slope AND curvature stress events (butterfly distortions)
- Expected autocorr near 0 (product of two near-iid series)
- Does NOT add another high-autocorr feature (avoids attempt 8's Sharpe degradation pattern)

**Self-contained**: Data fetch (FRED public CSV) + Feature computation + Gate evaluation + Save results

In [ ]:
# Cell 1: Install dependencies
import pandas as pd
import numpy as np
import json
import os
import warnings
import urllib.request
from io import StringIO
from datetime import datetime

warnings.filterwarnings('ignore')
print(f'Started: {datetime.now().isoformat()}')
print('Libraries loaded successfully')

In [ ]:
# Cell 2: FRED Public CSV Data Fetcher (No API key required)
def fetch_fred_series(series_id, start_date=None, end_date=None):
    """
    Fetch FRED series using public CSV endpoint (no API key required).
    Returns a pandas Series indexed by date.
    """
    url = f'https://fred.stlouisfed.org/graph/fredgraph.csv?id={series_id}'
    try:
        req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        with urllib.request.urlopen(req, timeout=60) as response:
            csv_data = response.read().decode('utf-8')
    except Exception as e:
        raise RuntimeError(f'Failed to fetch FRED series {series_id}: {e}')

    df = pd.read_csv(StringIO(csv_data))
    date_col = df.columns[0]
    val_col = df.columns[1]
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.set_index(date_col)
    series = pd.to_numeric(df[val_col], errors='coerce')
    series.name = series_id
    if start_date:
        series = series[series.index >= pd.Timestamp(start_date)]
    if end_date:
        series = series[series.index <= pd.Timestamp(end_date)]
    return series

print('FRED public CSV fetcher ready (no API key required)')

In [ ]:
# Cell 3: Data Fetching - DFII5, DFII10, DFII30
print('=' * 60)
print('DATA FETCHING')
print('=' * 60)

FETCH_START = '2014-01-01'
FETCH_END = '2025-02-28'
SCHEMA_START = '2015-01-02'
SCHEMA_END = '2025-02-12'

# All 3 DFII series needed for slope and curvature computation
series_ids = ['DFII5', 'DFII10', 'DFII30']
raw_series = {}
for sid in series_ids:
    s = fetch_fred_series(sid, start_date=FETCH_START, end_date=FETCH_END)
    raw_series[sid] = s
    n_clean = s.dropna()
    print(f'  {sid}: {len(n_clean)} obs, {n_clean.index[0].date()} to {n_clean.index[-1].date()}')

df_raw = pd.DataFrame(raw_series)
df_raw.index = pd.to_datetime(df_raw.index)
df_raw = df_raw.sort_index()

df = df_raw.dropna()
print(f'\nCombined (all 3 non-null): {len(df)} obs, {df.index[0].date()} to {df.index[-1].date()}')

In [ ]:
# Cell 4: Intermediate Series Computation
print('=' * 60)
print('INTERMEDIATE SERIES')
print('=' * 60)

df['slope'] = df['DFII30'] - df['DFII5']
df['curvature'] = 2 * df['DFII10'] - df['DFII5'] - df['DFII30']

print(f'Slope range: {df["slope"].min():.3f} to {df["slope"].max():.3f} (mean={df["slope"].mean():.3f})')
print(f'Curvature range: {df["curvature"].min():.3f} to {df["curvature"].max():.3f} (mean={df["curvature"].mean():.3f})')

df['dfii10_chg'] = df['DFII10'].diff()
df['slope_chg'] = df['slope'].diff()
df['curvature_chg'] = df['curvature'].diff()

# Interaction: product of slope_chg and curvature_chg
# Captures simultaneous butterfly distortion events
df['slope_curvature_interaction'] = df['slope_chg'] * df['curvature_chg']

print(f'DFII10 daily chg: [{df["dfii10_chg"].min():.4f}, {df["dfii10_chg"].max():.4f}]')
print(f'Slope daily chg: [{df["slope_chg"].min():.4f}, {df["slope_chg"].max():.4f}]')
print(f'Curvature daily chg: [{df["curvature_chg"].min():.4f}, {df["curvature_chg"].max():.4f}]')
print(f'Slope*Curvature interaction: [{df["slope_curvature_interaction"].min():.6f}, {df["slope_curvature_interaction"].max():.6f}]')

In [ ]:
# Cell 5: Submodel Dataset Path Resolution
print('=' * 60)
print('DATASET PATH RESOLUTION')
print('=' * 60)

candidate_paths = [
    '/kaggle/input/datasets/bigbigzabuton/gold-prediction-submodels',
    '../input/gold-prediction-submodels',
    '/kaggle/input/gold-prediction-submodels',
]
SUBMODEL_PATH = None
for cp in candidate_paths:
    if os.path.exists(cp):
        SUBMODEL_PATH = cp
        print(f'Found submodel dataset at: {cp}')
        break

if SUBMODEL_PATH is None:
    print('ERROR: gold-prediction-submodels not found!')
    for root in ['/kaggle/input', '../input']:
        if os.path.exists(root):
            print(f'Available at {root}: {os.listdir(root)}')
    raise RuntimeError('gold-prediction-submodels dataset not found. Check kernel-metadata.json dataset_sources.')

submodel_files = os.listdir(SUBMODEL_PATH)
print(f'Submodel files ({len(submodel_files)}): {submodel_files[:10]}')

In [ ]:
# Cell 6: Feature Computation (Deterministic - 5 features)
print('=' * 60)
print('FEATURE COMPUTATION (ATTEMPT 9: 4 ATTEMPT-7 FEATURES + 1 INTERACTION)')
print('=' * 60)
print('Design:')
print('  Features 1-4: same as attempt 7 (rr_level_change_z, rr_slope_chg_z,')
print('                rr_curvature_chg_z, rr_slope_level_z)')
print('  Feature 5 (NEW): rr_slope_curvature_interaction_z')
print('    = rolling_zscore(slope.diff() * curvature.diff(), 60)')
print('    Captures butterfly distortion events, low autocorr, orthogonal to existing')
print()

def rolling_zscore(series, window, min_periods_ratio=0.5):
    """Rolling z-score normalization. Returns nan during warmup."""
    min_p = max(10, int(window * min_periods_ratio))
    mu = series.rolling(window, min_periods=min_p).mean()
    sigma = series.rolling(window, min_periods=min_p).std()
    sigma = sigma.where(sigma > 1e-10, np.nan)
    z = (series - mu) / sigma
    return z.clip(-4, 4)

# Feature 1: rr_level_change_z - standardized daily change in 10Y real yield
# Window=30: captures ~1.5 month regime of rate velocity
rr_level_change_z = rolling_zscore(df['dfii10_chg'], 30)

# Feature 2: rr_slope_chg_z - standardized daily change in slope (30Y-5Y)
# Window=60: captures medium-term steepening/flattening dynamics
rr_slope_chg_z = rolling_zscore(df['slope_chg'], 60)

# Feature 3: rr_curvature_chg_z - standardized daily change in curvature
# Window=60: captures belly distortion dynamics
rr_curvature_chg_z = rolling_zscore(df['curvature_chg'], 60)

# Feature 4: rr_slope_level_z - current slope level (regime indicator)
# Re-included from attempt 7. Critical for Sharpe improvement (+0.329 in attempt 7).
# Removal in attempt 8 reversed Sharpe to -0.260.
rr_slope_level_z = rolling_zscore(df['slope'], 60)

# Feature 5 (NEW): rr_slope_curvature_interaction_z
# Product of slope_chg and curvature_chg: captures butterfly distortion events
# when BOTH slope AND curvature are simultaneously stressed.
# Low autocorr (product of two near-iid series) - will not degrade MAE like rr_slope_level_z
# Orthogonal to individual features by construction of cross-products
rr_slope_curvature_interaction_z = rolling_zscore(df['slope_curvature_interaction'], 60)

features = pd.DataFrame({
    'rr_level_change_z': rr_level_change_z,
    'rr_slope_chg_z': rr_slope_chg_z,
    'rr_curvature_chg_z': rr_curvature_chg_z,
    'rr_slope_level_z': rr_slope_level_z,
    'rr_slope_curvature_interaction_z': rr_slope_curvature_interaction_z,
}, index=df.index)

print('Features computed (5 total):')
for col in features.columns:
    n_valid = features[col].notna().sum()
    n_nan = features[col].isna().sum()
    nan_pct = n_nan / len(features) * 100
    ac = features[col].autocorr(1)
    std = features[col].std()
    print(f'  {col}: valid={n_valid}, nan={n_nan} ({nan_pct:.1f}%), autocorr={ac:.4f}, std={std:.4f}')

In [ ]:
# Cell 7: Align to Schema Date Range
print('=' * 60)
print('ALIGNMENT TO SCHEMA DATES')
print('=' * 60)

import yfinance as yf
gold = yf.download('GC=F', start=FETCH_START, end=FETCH_END, auto_adjust=True, progress=False)
gold_dates = gold.index
print(f'Gold trading calendar: {len(gold_dates)} dates, {gold_dates[0].date()} to {gold_dates[-1].date()}')

schema_dates = gold_dates[(gold_dates >= SCHEMA_START) & (gold_dates <= SCHEMA_END)]
print(f'Schema dates: {len(schema_dates)} dates, {schema_dates[0].date()} to {schema_dates[-1].date()}')

features_aligned = features.reindex(schema_dates, method='ffill', limit=1)

gold_close = gold['Close'].squeeze()
gold_return = gold_close.pct_change() * 100
gold_return_next = gold_return.shift(-1)
gold_return_next_schema = gold_return_next.reindex(schema_dates)

print(f'Features aligned shape: {features_aligned.shape}')
print(f'NaN counts after alignment:')
for col in features_aligned.columns:
    n_nan = features_aligned[col].isna().sum()
    print(f'  {col}: {n_nan} NaN ({n_nan/len(features_aligned)*100:.1f}%)')

In [ ]:
# Cell 8: Data Split (70/15/15)
print('=' * 60)
print('DATA SPLIT')
print('=' * 60)

common_mask = features_aligned.notna().all(axis=1) & gold_return_next_schema.notna()
common_dates = schema_dates[common_mask]
print(f'Common dates (all 5 features + target): {len(common_dates)}')

n = len(common_dates)
n_train = int(n * 0.70)
n_val = int(n * 0.15)
n_test = n - n_train - n_val

train_dates = common_dates[:n_train]
val_dates = common_dates[n_train:n_train + n_val]
test_dates = common_dates[n_train + n_val:]

print(f'Train: {len(train_dates)} ({train_dates[0].date()} to {train_dates[-1].date()})')
print(f'Val:   {len(val_dates)} ({val_dates[0].date()} to {val_dates[-1].date()})')
print(f'Test:  {len(test_dates)} ({test_dates[0].date()} to {test_dates[-1].date()})')

In [ ]:
# Cell 9: Gate 1 - Standalone Quality Check
print('=' * 60)
print('GATE 1: STANDALONE QUALITY')
print('=' * 60)

gate1_results = {}

for col in features_aligned.columns:
    feat = features_aligned[col].dropna()
    nan_pct = features_aligned[col].isna().mean() * 100
    std_val = feat.std()
    autocorr = feat.autocorr(1)

    is_constant = std_val < 0.01
    is_all_nan = len(feat) == 0
    high_autocorr = autocorr > 0.99

    pass_gate1 = not is_constant and not is_all_nan and not high_autocorr

    gate1_results[col] = {
        'nan_pct': float(nan_pct),
        'std': float(std_val),
        'autocorr': float(autocorr),
        'is_constant': bool(is_constant),
        'is_all_nan': bool(is_all_nan),
        'high_autocorr': bool(high_autocorr),
        'pass': bool(pass_gate1)
    }

    status = 'PASS' if pass_gate1 else 'FAIL'
    print(f'  {col}: {status} | nan={nan_pct:.1f}%, std={std_val:.4f}, autocorr={autocorr:.4f}')

gate1_pass = all(r['pass'] for r in gate1_results.values())

overfit_ratio = 1.0  # deterministic features, no training
print(f'\nOverfit ratio: {overfit_ratio:.2f} (deterministic, no training)')
print(f'Gate 1 Overall: {"PASS" if gate1_pass else "FAIL"}')

In [ ]:
# Cell 10: Gate 2 - Information Gain (MI test)
print('=' * 60)
print('GATE 2: INFORMATION GAIN')
print('=' * 60)

from sklearn.metrics import mutual_info_score

def compute_mi(feature, target, n_bins=20):
    """Compute mutual information using quantile binning."""
    mask = feature.notna() & target.notna()
    f = feature[mask]
    t = target[mask]
    if len(f) < 100:
        return 0.0
    try:
        f_binned = pd.qcut(f, q=n_bins, labels=False, duplicates='drop')
        t_binned = pd.qcut(t, q=n_bins, labels=False, duplicates='drop')
        return float(mutual_info_score(f_binned, t_binned))
    except Exception:
        return 0.0

base_mi_total = None
try:
    base_path = os.path.join(SUBMODEL_PATH, 'base_features.csv')
    if not os.path.exists(base_path):
        base_path = os.path.join(SUBMODEL_PATH, 'meta_model_input.csv')
    base_df = pd.read_csv(base_path, index_col=0, parse_dates=True)
    base_df.index = pd.to_datetime(base_df.index)
    print(f'Loaded base features: {base_df.shape}')

    target_test = gold_return_next_schema.reindex(test_dates)
    base_test = base_df.reindex(test_dates)

    base_mi_total = 0.0
    for col in base_df.select_dtypes(include=[np.number]).columns[:10]:
        mi = compute_mi(base_test[col], target_test)
        base_mi_total += mi
    print(f'Baseline MI (top 10 base features, test set): {base_mi_total:.6f}')
except Exception as e:
    print(f'WARNING: Could not load base features: {e}')
    base_mi_total = 0.1

target_test = gold_return_next_schema.reindex(test_dates)
feat_test = features_aligned.reindex(test_dates)

new_mi_total = 0.0
per_feature_mi = {}
for col in features_aligned.columns:
    mi = compute_mi(feat_test[col], target_test)
    new_mi_total += mi
    per_feature_mi[col] = float(mi)
    print(f'  MI({col}) = {mi:.6f}')

print(f'\nNew features MI total: {new_mi_total:.6f}')
print(f'Baseline MI total: {base_mi_total:.6f}')

if base_mi_total > 0:
    mi_increase_pct = (new_mi_total / base_mi_total) * 100
else:
    mi_increase_pct = 999.0
print(f'MI increase: {mi_increase_pct:.2f}% (threshold: > 5%)')
gate2_mi_pass = mi_increase_pct > 5.0
print(f'Gate 2 MI: {"PASS" if gate2_mi_pass else "FAIL"}')

from numpy.linalg import inv
feat_clean = feat_test.dropna()
if len(feat_clean) > 10:
    X = feat_clean.values
    corr = np.corrcoef(X.T)
    try:
        vif_values = np.diag(inv(corr))
        max_vif = float(np.max(vif_values))
    except Exception:
        max_vif = 999.0
else:
    max_vif = 0.0
print(f'Max VIF: {max_vif:.3f} (threshold: < 10)')
gate2_vif_pass = max_vif < 10.0

target_full = gold_return_next_schema
rolling_corr_stds = []
for col in features_aligned.columns:
    merged = pd.concat([features_aligned[col], target_full], axis=1).dropna()
    if len(merged) > 120:
        rc = merged.iloc[:,0].rolling(60).corr(merged.iloc[:,1])
        rolling_corr_stds.append(float(rc.std()))
max_rolling_corr_std = max(rolling_corr_stds) if rolling_corr_stds else 0.0
print(f'Max rolling corr std: {max_rolling_corr_std:.4f} (threshold: < 0.15)')
gate2_stability_pass = max_rolling_corr_std < 0.15

gate2_pass = gate2_mi_pass and gate2_vif_pass
print(f'Gate 2 Overall: {"PASS" if gate2_pass else "FAIL"} (MI={gate2_mi_pass}, VIF={gate2_vif_pass}, stability={gate2_stability_pass})')

In [ ]:
# Cell 11: Gate 3 - Ablation Test (4-fold cross-validation)
print('=' * 60)
print('GATE 3: ABLATION TEST (4-FOLD CV)')
print('=' * 60)

import xgboost as xgb
from sklearn.metrics import mean_absolute_error

meta_input_path = os.path.join(SUBMODEL_PATH, 'meta_model_input.csv')
if os.path.exists(meta_input_path):
    meta_df = pd.read_csv(meta_input_path, index_col=0, parse_dates=True)
    print(f'Meta model input loaded: {meta_df.shape}')
else:
    print('meta_model_input.csv not found - constructing from base_features_raw.csv + submodel CSVs')
    base_path = os.path.join(SUBMODEL_PATH, 'base_features_raw.csv')
    meta_df = pd.read_csv(base_path, index_col=0, parse_dates=True)
    print(f'  base_features_raw.csv: {meta_df.shape}')
    sm_files = sorted([f for f in os.listdir(SUBMODEL_PATH)
                       if f.endswith('.csv') and f != 'base_features_raw.csv'
                       and 'real_rate' not in f])
    for fname in sm_files:
        try:
            sm_df = pd.read_csv(os.path.join(SUBMODEL_PATH, fname), index_col=0, parse_dates=True)
            sm_df.index = pd.to_datetime(sm_df.index)
            meta_df = meta_df.join(sm_df, how='left')
            print(f'  {fname}: {sm_df.shape}')
        except Exception as e:
            print(f'  WARNING: Could not load {fname}: {e}')
    print(f'Meta model input (constructed): {meta_df.shape}')

meta_df.index = pd.to_datetime(meta_df.index)
print(f'Columns: {list(meta_df.columns[:10])}...')

target_col = 'gold_return_next' if 'gold_return_next' in meta_df.columns else meta_df.columns[0]
print(f'Target column: {target_col}')

base_feature_cols = [c for c in meta_df.columns if c != target_col]
print(f'Base feature columns: {len(base_feature_cols)}')

def compute_direction_accuracy(y_true, y_pred):
    mask = y_true != 0
    if mask.sum() == 0:
        return 0.5
    return float((np.sign(y_pred[mask]) == np.sign(y_true[mask])).mean())

def compute_sharpe(y_true, y_pred, cost_bps=5):
    position = np.sign(y_pred)
    strategy_ret = position * y_true
    position_changes = np.abs(np.diff(np.concatenate([[0], position])))
    costs = position_changes * (cost_bps / 10000.0)
    net_ret = strategy_ret - costs
    if net_ret.std() < 1e-10:
        return 0.0
    return float(net_ret.mean() / net_ret.std() * np.sqrt(252))

xgb_params = {
    'max_depth': 3,
    'learning_rate': 0.05,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_lambda': 1.0,
    'random_state': 42,
    'n_jobs': -1,
    'verbosity': 0
}

folds = []
n_common = len(common_dates)
fold_size = n_common // 4
for i in range(4):
    test_start = i * fold_size
    test_end = (i + 1) * fold_size if i < 3 else n_common
    train_idx = list(range(0, test_start))
    test_idx = list(range(test_start, test_end))
    if len(train_idx) < 50:
        print(f'  Fold {i+1}: skipped (insufficient training data)')
        continue
    folds.append((train_idx, test_idx))

print(f'Running {len(folds)} folds...')

base_da_folds = []
base_sharpe_folds = []
base_mae_folds = []
ext_da_folds = []
ext_sharpe_folds = []
ext_mae_folds = []

for fold_i, (train_idx, test_idx) in enumerate(folds):
    fold_train_dates = common_dates[train_idx]
    fold_test_dates = common_dates[test_idx]

    base_train = meta_df[base_feature_cols].reindex(fold_train_dates)
    base_test_fold = meta_df[base_feature_cols].reindex(fold_test_dates)
    y_train = meta_df[target_col].reindex(fold_train_dates)
    y_test = meta_df[target_col].reindex(fold_test_dates)

    new_train = features_aligned.reindex(fold_train_dates)
    new_test = features_aligned.reindex(fold_test_dates)

    ext_train = pd.concat([base_train, new_train], axis=1)
    ext_test = pd.concat([base_test_fold, new_test], axis=1)

    base_train_clean = base_train.join(y_train).dropna()
    ext_train_clean = ext_train.join(y_train).dropna()
    base_test_clean = base_test_fold.join(y_test).dropna()
    ext_test_clean = ext_test.join(y_test).dropna()

    if len(base_train_clean) < 50 or len(base_test_clean) < 10:
        print(f'  Fold {fold_i+1}: skipped (insufficient data)')
        continue

    base_model = xgb.XGBRegressor(**xgb_params)
    base_model.fit(base_train_clean[base_feature_cols], base_train_clean[target_col])
    base_pred = base_model.predict(base_test_clean[base_feature_cols])
    y_test_base = base_test_clean[target_col].values

    base_da = compute_direction_accuracy(y_test_base, base_pred)
    base_sh = compute_sharpe(y_test_base, base_pred)
    base_mae_val = mean_absolute_error(y_test_base, base_pred)
    base_da_folds.append(base_da)
    base_sharpe_folds.append(base_sh)
    base_mae_folds.append(base_mae_val)

    ext_cols = [c for c in ext_train.columns if c != target_col]
    ext_model = xgb.XGBRegressor(**xgb_params)
    ext_model.fit(ext_train_clean[ext_cols], ext_train_clean[target_col])
    ext_pred = ext_model.predict(ext_test_clean[ext_cols])
    y_test_ext = ext_test_clean[target_col].values

    ext_da = compute_direction_accuracy(y_test_ext, ext_pred)
    ext_sh = compute_sharpe(y_test_ext, ext_pred)
    ext_mae_val = mean_absolute_error(y_test_ext, ext_pred)
    ext_da_folds.append(ext_da)
    ext_sharpe_folds.append(ext_sh)
    ext_mae_folds.append(ext_mae_val)

    print(f'  Fold {fold_i+1}: Base DA={base_da:.4f}, Ext DA={ext_da:.4f} (delta={ext_da-base_da:+.4f})')
    print(f'           Base Sharpe={base_sh:.4f}, Ext Sharpe={ext_sh:.4f} (delta={ext_sh-base_sh:+.4f})')
    print(f'           Base MAE={base_mae_val:.4f}, Ext MAE={ext_mae_val:.4f} (delta={ext_mae_val-base_mae_val:+.4f})')

base_da_avg = float(np.mean(base_da_folds))
ext_da_avg = float(np.mean(ext_da_folds))
base_sharpe_avg = float(np.mean(base_sharpe_folds))
ext_sharpe_avg = float(np.mean(ext_sharpe_folds))
base_mae_avg = float(np.mean(base_mae_folds))
ext_mae_avg = float(np.mean(ext_mae_folds))

da_delta = ext_da_avg - base_da_avg
sharpe_delta = ext_sharpe_avg - base_sharpe_avg
mae_delta = ext_mae_avg - base_mae_avg

print(f'\n--- Gate 3 Summary ---')
print(f'Direction Accuracy: {base_da_avg:.4f} -> {ext_da_avg:.4f} (delta={da_delta:+.4f})')
print(f'Sharpe Ratio:       {base_sharpe_avg:.4f} -> {ext_sharpe_avg:.4f} (delta={sharpe_delta:+.4f})')
print(f'MAE:                {base_mae_avg:.4f} -> {ext_mae_avg:.4f} (delta={mae_delta:+.4f})')

gate3_da_pass = da_delta >= 0.005
gate3_sharpe_pass = sharpe_delta >= 0.05
gate3_mae_pass = mae_delta <= -0.01
gate3_pass = gate3_da_pass or gate3_sharpe_pass or gate3_mae_pass
print(f'Gate 3: DA={gate3_da_pass}, Sharpe={gate3_sharpe_pass}, MAE={gate3_mae_pass}')
print(f'Gate 3 Overall: {"PASS" if gate3_pass else "FAIL"}')

In [ ]:
# Cell 12: Save Outputs
print('=' * 60)
print('SAVING OUTPUTS')
print('=' * 60)

# 1. Submodel output CSV (5 columns)
output = features_aligned.copy()
output.index.name = 'date'
output.to_csv('submodel_output.csv')
print(f'Saved submodel_output.csv: {output.shape}')
print(f'Columns: {list(output.columns)}')

# 2. Training result JSON
result = {
    'feature': 'real_rate',
    'attempt': 9,
    'method': 'deterministic_yield_curve_5feature_with_interaction',
    'timestamp': datetime.now().isoformat(),
    'description': 'Real yield curve: 4 attempt-7 features retained + slope-curvature interaction z-score as 5th feature. Attempt 7 achieved Sharpe +0.329 via rr_slope_level_z. Attempt 8 removed it but Sharpe reversed to -0.260 (MAE improved). Attempt 9 retains rr_slope_level_z + adds low-autocorr interaction feature to push DA positive.',
    'design_choices': {
        'retained_from_attempt_7': ['rr_level_change_z', 'rr_slope_chg_z', 'rr_curvature_chg_z', 'rr_slope_level_z'],
        'new_in_attempt_9': 'rr_slope_curvature_interaction_z = rolling_zscore(slope.diff() * curvature.diff(), 60)',
        'rationale': 'Interaction feature captures butterfly distortion events. Low autocorr (product of two near-iid series). Orthogonal to existing features.'
    },
    'data_sources': ['FRED:DFII5', 'FRED:DFII10', 'FRED:DFII30'],
    'output_shape': list(output.shape),
    'output_columns': list(output.columns),
    'gate1': {
        'pass': gate1_pass,
        'overfit_ratio': overfit_ratio,
        'checks': gate1_results
    },
    'gate2': {
        'pass': gate2_pass,
        'checks': {
            'mi': {
                'new_total': float(new_mi_total),
                'baseline_total': float(base_mi_total),
                'increase': float(mi_increase_pct),
                'pass': bool(gate2_mi_pass)
            },
            'vif': {
                'max': float(max_vif),
                'pass': bool(gate2_vif_pass)
            },
            'stability': {
                'max_rolling_corr_std': float(max_rolling_corr_std),
                'pass': bool(gate2_stability_pass)
            }
        },
        'per_feature_mi': per_feature_mi
    },
    'gate3': {
        'pass': gate3_pass,
        'baseline': {
            'direction_accuracy': base_da_avg,
            'sharpe_ratio': base_sharpe_avg,
            'mae': base_mae_avg
        },
        'extended': {
            'direction_accuracy': ext_da_avg,
            'sharpe_ratio': ext_sharpe_avg,
            'mae': ext_mae_avg
        },
        'checks': {
            'direction': {
                'delta': float(da_delta),
                'threshold': 0.005,
                'pass': bool(gate3_da_pass)
            },
            'sharpe': {
                'delta': float(sharpe_delta),
                'threshold': 0.05,
                'pass': bool(gate3_sharpe_pass)
            },
            'mae': {
                'delta': float(mae_delta),
                'threshold': -0.01,
                'pass': bool(gate3_mae_pass)
            }
        },
        'fold_results': {
            'base_da': base_da_folds,
            'ext_da': ext_da_folds,
            'base_sharpe': base_sharpe_folds,
            'ext_sharpe': ext_sharpe_folds,
            'base_mae': base_mae_folds,
            'ext_mae': ext_mae_folds
        }
    },
    'feature_stats': {
        col: {
            'mean': float(output[col].mean()),
            'std': float(output[col].std()),
            'min': float(output[col].min()),
            'max': float(output[col].max()),
            'autocorr': float(output[col].autocorr(1)),
            'nan_pct': float(output[col].isna().mean() * 100)
        }
        for col in output.columns
    }
}

with open('training_result.json', 'w') as f:
    json.dump(result, f, indent=2)
print('Saved training_result.json')

print(f'\nFinished: {datetime.now().isoformat()}')
print('Training complete!')
print(f'Gate 1: {"PASS" if gate1_pass else "FAIL"}')
print(f'Gate 2: {"PASS" if gate2_pass else "FAIL"} (MI +{mi_increase_pct:.1f}%, VIF {max_vif:.2f})')
print(f'Gate 3: {"PASS" if gate3_pass else "FAIL"} (DA {da_delta:+.4f}, Sharpe {sharpe_delta:+.4f}, MAE {mae_delta:+.4f})')